In [1]:
import os
import tempfile
from rdkit import Chem
from rdkit.Chem import AllChem
from druglab.cuby4 import (
    Cuby4Config,
    Cuby4MOPACInterfaceConfig,
    Cuby4AMBERInterfaceConfig,
    Cuby4QMMMInterfaceConfig,
    Cuby4MOPACEnergyCalculator,
    Cuby4MOPACEnergyOptimizer,
    Cuby4AMBEREnergyCalculator,
    Cuby4AMBEREnergyOptimizer,
    Cuby4AMBERMultiComponentEnergyOptimizer,
    Cuby4QMMMEnergyCalculator,
    Cuby4QMMMEnergyOptimizer
)

def create_test_molecule(smiles="CC"):
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol)
    AllChem.MMFFOptimizeMolecule(mol)
    return mol

os.mkdir("/mnt/d/druglab/druglab_forked/druglab/notebooks/test")
work_dir = "/mnt/d/druglab/druglab_forked/druglab/notebooks/test"

test_molecule = create_test_molecule()
test_molecule2 = create_test_molecule("CCO")
test_qm_mol = create_test_molecule("CC")
test_mm_mol = create_test_molecule("O")

print("Testing MOPAC Interface Config")
mopac_interface_config = Cuby4MOPACInterfaceConfig(
    method="pm7",
    mozyme=True,
    keywords=["PRECISE", "NODIIS"]
)
print(mopac_interface_config.get_string())

print("\nTesting MOPAC Energy Calculator")
mopac_calculator = Cuby4MOPACEnergyCalculator(
    interface_config=mopac_interface_config,
    work_dir=work_dir
)
input_dict = {"molecules": test_molecule}
job_config = mopac_calculator.generate_job_config(input_dict)
print(job_config.get_string())

print("\nTesting MOPAC Energy Optimizer")
mopac_optimizer = Cuby4MOPACEnergyOptimizer(
    interface_config=mopac_interface_config,
    work_dir=work_dir,
    max_cycles=5
)
job_config = mopac_optimizer.generate_job_config(input_dict)
print(job_config.get_string())

print("\nTesting AMBER Interface Config")
amber_interface_config = Cuby4AMBERInterfaceConfig(
    home="/home/arian/mambaforge/envs/druglab"
)
print(amber_interface_config.get_string())

print("\nTesting AMBER Energy Calculator")
amber_calculator = Cuby4AMBEREnergyCalculator(
    interface_config=amber_interface_config,
    work_dir=work_dir
)
job_config = amber_calculator.generate_job_config(input_dict)
print(job_config.get_string())

print("\nTesting AMBER Energy Optimizer")
amber_optimizer = Cuby4AMBEREnergyOptimizer(
    interface_config=amber_interface_config,
    work_dir=work_dir
)
job_config = amber_optimizer.generate_job_config(input_dict)
print(job_config.get_string())

print("\nTesting Multi-Component AMBER Optimizer")
multi_input = {
    "molecule_1": test_molecule,
    "molecule_2": test_molecule2
}
amber_multi_optimizer = Cuby4AMBERMultiComponentEnergyOptimizer(
    interface_config=amber_interface_config,
    work_dir=work_dir,
    num_components=2
)
job_config = amber_multi_optimizer.generate_job_config(multi_input)
print(job_config.get_string())

print("\nTesting QM/MM Interface Config")
qm_config = Cuby4MOPACInterfaceConfig(method="pm7")
mm_config = Cuby4AMBERInterfaceConfig()
qmmm_interface_config = Cuby4QMMMInterfaceConfig(
    qm_config=qm_config,
    mm_config=mm_config,
    embedding="electrostatic"
)
print(qmmm_interface_config.get_string())

print("\nTesting QM/MM Energy Calculator")
qmmm_input = {
    "qm_region": test_qm_mol,
    "nonqm_region": test_mm_mol
}
qmmm_calculator = Cuby4QMMMEnergyCalculator(
    interface_config=qmmm_interface_config,
    work_dir=work_dir
)
job_config = qmmm_calculator.generate_job_config(qmmm_input)
print(job_config.get_string())

print("\nTesting QM/MM Energy Optimizer")
qmmm_optimizer = Cuby4QMMMEnergyOptimizer(
    interface_config=qmmm_interface_config,
    work_dir=work_dir,
    maxcycles=10
)
job_config = qmmm_optimizer.generate_job_config(qmmm_input)
print(job_config.get_string())

print("\nTesting calculations (if software installed)")
try:
    print("\nTrying MOPAC calculation")
    result = mopac_calculator.operate(input_dict)
    print(f"MOPAC energy: {result['energy']}")
except Exception as e:
    print(f"MOPAC calculation failed: {str(e)}")

try:
    print("\nTrying AMBER calculation")
    result = amber_calculator.operate(input_dict)
    print(f"AMBER energy: {result['energy']}")
except Exception as e:
    print(f"AMBER calculation failed: {str(e)}")

try:
    print("\nTrying QM/MM calculation")
    result = qmmm_calculator.operate(qmmm_input)
    print(f"QM/MM energy: {result['energy']}")
except Exception as e:
    print(f"QM/MM calculation failed: {str(e)}")

print("\nCleaning up temporary files")
# for f in os.listdir(work_dir):
#     os.remove(os.path.join(work_dir, f))
# os.rmdir(work_dir)

print("\nAll tests completed")
# TODO see yaml files and correct based on them

#for mopac: add interface


Testing MOPAC Interface Config
cuby_threads: 1
mopac_keywords: PRECISE NODIIS
mopac_exe: /home/arian/mambaforge/envs/ASH/bin/mopac
method: pm7
mopac_mozyme: yes


Testing MOPAC Energy Calculator
job: energy
geometry: /mnt/d/druglab/druglab_forked/druglab/notebooks/test/nz59qf_c4mecinp.pdb
charge: 0
multiplicity: 1
interface: mopac


Testing MOPAC Energy Optimizer
job: optimize
geometry: /mnt/d/druglab/druglab_forked/druglab/notebooks/test/wuuzsl_c4meoinp.pdb
charge: 0
multiplicity: 1
restart_file: /mnt/d/druglab/druglab_forked/druglab/notebooks/test/09s9rt_c4meoout.pdb
maxcycles: 5
interface: mopac


Testing AMBER Interface Config
cuby_threads: 1
amber_amberhome: /home/arian/mambaforge/envs/druglab


Testing AMBER Energy Calculator
job: energy
geometry: /mnt/d/druglab/druglab_forked/druglab/notebooks/test/5u8y9s.pdb
charge: 0
multiplicity: 1
amber_top_file: /mnt/d/druglab/druglab_forked/druglab/notebooks/test/ka833b.parm7
interface: amber


Testing AMBER Energy Optimizer
job: optimize


/home/arian/mambaforge/envs/druglab/lib/python3.12/site-packages/openff/toolkit/topology/molecule.py:4387: MultipleComponentsInMoleculeWarning: RDKit Molecule passed to from_rdkit consists of more than one molecule, consider running rdkit.Chem.AllChem.GetMolFrags(rdmol, asMols=True) or splitting input SMILES at '.' to get separate molecules and pass them to from_rdkit one at a time. While this is supported for legacy reasons, OpenFF Molecule objects are not supposed to contain disconnected chemical graphs and this may result in undefined behavior later on. The OpenFF ecosystem is built to handle multiple molecules, but they should be in a Topology object, ex: top = Topology.from_molecules([mol1, mol2])
  molecule = toolkit.from_rdkit(


job: energy
geometry: /mnt/d/druglab/druglab_forked/druglab/notebooks/test/hwe8i3.pdb
charge: 0
multiplicity: 1
qmmm_core: 1-8
calculation_qm:
  interface: mopac
  charge: 0
calculation_mm:
  interface: amber
  amber_top_file: /mnt/d/druglab/druglab_forked/druglab/notebooks/test/oidhr5.parm7
calculation_qmregion_mm:
  amber_top_file: /mnt/d/druglab/druglab_forked/druglab/notebooks/test/lw562t.parm7
interface: qmmm


Testing QM/MM Energy Optimizer
job: optimize
geometry: /mnt/d/druglab/druglab_forked/druglab/notebooks/test/ah4g14.pdb
charge: 0
multiplicity: 1
restart_file: /mnt/d/druglab/druglab_forked/druglab/notebooks/test/hbbc2o.pdb
maxcycles: 10
qmmm_core: 1-8
calculation_qm:
  charge: 0
  interface: mopac
calculation_mm:
  interface: amber
  amber_top_file: /mnt/d/druglab/druglab_forked/druglab/notebooks/test/fud7b4.parm7
calculation_qmregion_mm:
  amber_top_file: /mnt/d/druglab/druglab_forked/druglab/notebooks/test/hyc0nc.parm7
interface: qmmm


Testing calculations (if software i